Downloads

In [1]:
!pip install ultralytics
!pip install opencv-python
!pip install supervision

Imports

In [1]:
import os
from ultralytics import YOLO
#from google.colab import files
import supervision as sv
import cv2
import json
import numpy as np
from tqdm import tqdm
from sports.common.team import TeamClassifier

from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image

#SigLIP Call
clip_model = CLIPModel.from_pretrained("google/siglip-base-patch16-224")
clip_processor = CLIPProcessor.from_pretrained("google/siglip-base-patch16-224")



You are using a model of type siglip to instantiate a model of type clip. This is not supported for all configurations of models and can yield errors.


RuntimeError: Error(s) in loading state_dict for CLIPModel:
	size mismatch for text_model.embeddings.position_embedding.weight: copying a param with shape torch.Size([64, 768]) from checkpoint, the shape in current model is torch.Size([77, 768]).
	size mismatch for vision_model.embeddings.position_embedding.weight: copying a param with shape torch.Size([196, 768]) from checkpoint, the shape in current model is torch.Size([197, 768]).
	You may consider adding `ignore_mismatched_sizes=True` in the model `from_pretrained` method.

In [ ]:
#uploaded = files.upload()


Saving 121364_0.mp4 to 121364_0.mp4


In [5]:
player_model = YOLO('best.pt')

player_model.val(data="C:/Users/Yahya/Desktop/football_analysis-main/Nujoom/football-players-detection.v12i.yolov8/data.yaml", split='test')


Ultralytics 8.3.113  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 112 layers, 68,127,420 parameters, 0 gradients, 257.4 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 334.122.5 MB/s, size: 182.0 KB)


val: Scanning C:\Users\Yahya\Desktop\football_analysis-main\Nujoom\football-players-detection.v12i.yolov8\test\labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [01:00<00:00, 30.08s/it]


                   all         25        598      0.934      0.828      0.875      0.689
                  ball         23         24          1      0.486       0.65      0.395
            goalkeeper         19         19      0.847      0.875      0.898      0.763
                player         25        499      0.977      0.988      0.994      0.864
               referee         25         56      0.911      0.964      0.959      0.732
Speed: 23.7ms preprocess, 1979.6ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to runs\detect\val17


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002A4C24EAF00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
       

In [9]:
ball_model = YOLO('ball_best.pt')

ball_model.val(data="C:/Users/Yahya/Desktop/football_analysis-main/Nujoom/football-ball-detection.v4i.yolov8/data.yaml", split='test')



Ultralytics 8.3.113  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 5.92.2 MB/s, size: 39.9 KB)


val: Scanning C:\Users\Yahya\Desktop\football_analysis-main\Nujoom\football-ball-detection.v4i.yolov8\test\labels... 500 images, 372 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 633.72it/s]

val: New cache created: C:\Users\Yahya\Desktop\football_analysis-main\Nujoom\football-ball-detection.v4i.yolov8\test\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:13<00:00,  2.29it/s]


                   all        500        128       0.88      0.836      0.915      0.563
                  ball        128        128       0.88      0.836      0.915      0.563
Speed: 4.5ms preprocess, 12.4ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs\detect\val6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000023849C57770>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:
SOURCE_VIDEO_PATH = "121364_0.mp4"
OUTPUT_VIDEO_PATH = "121364_0_annotated.mp4"
JSON_OUTPUT_PATH = "121364_0_log.json"
BALL_ID = 0
GOALKEEPER_CLASS_ID = 1  # assume class ID 1 is the goalkeeper

# Annotators
ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)

# Tracker
tracker = sv.ByteTrack()
tracker.reset()

# Video setup
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
log_data = []

for frame_index, frame in tqdm(enumerate(frame_generator), total=total_frames):
    result = model.predict(frame, conf=0.1)[0]
    detections = sv.Detections.from_ultralytics(result)

    ball_detections = detections[detections.class_id == BALL_ID]
    if len(ball_detections) > 1:
        ball_detections.confidence = ball_detections.confidence.tolist()
        top_idx = np.argmax(ball_detections.confidence)
        ball_detections.xyxy = np.array([ball_detections.xyxy[top_idx]])
    ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

    goalkeeper_detections = detections[detections.class_id == GOALKEEPER_CLASS_ID]
    if len(goalkeeper_detections) > 1:
        goalkeeper_detections.confidence = goalkeeper_detections.confidence.tolist()
        top_idx = np.argmax(goalkeeper_detections.confidence)
        goalkeeper_detections.xyxy = np.array([goalkeeper_detections.xyxy[top_idx]])

    other_detections = detections[(detections.class_id != BALL_ID) & (detections.class_id != GOALKEEPER_CLASS_ID)]
    all_detections = other_detections.with_nms(threshold=0.5, class_agnostic=True)
    all_detections.class_id -= 1
    all_detections = tracker.update_with_detections(detections=all_detections)

    # Spliting the teams using SigLIP classification
    team_prompts = ["player wearing red team jersey", "player wearing blue team jersey"]

    team_assignments = {}
    for i, (tracker_id, box) in enumerate(zip(all_detections.tracker_id, all_detections.xyxy)):
        x1, y1, x2, y2 = map(int, box)
        player_crop = frame[y1:y2, x1:x2]
        if player_crop.size == 0:
            continue

        image_pil = Image.fromarray(player_crop)
        inputs = clip_processor(text=team_prompts, images=image_pil, return_tensors="pt", padding=True)
        with torch.no_grad():
            outputs = clip_model(**inputs)
            logits_per_image = outputs.logits_per_image
            probs = logits_per_image.softmax(dim=1)
            team_id = int(torch.argmax(probs).item())

        team_assignments[int(tracker_id)] = team_id

    #GoalKeeper team
    goalkeeper_team_id = None
    # if goalkeeper_detections.xyxy is not None and len(goalkeeper_detections.xyxy) > 0:
    #     gx1, gy1, gx2, gy2 = map(int, goalkeeper_detections.xyxy[0])
    #     gk_crop = frame[gy1:gy2, gx1:gx2]
    #     if gk_crop.size != 0:
    #         gk_color = gk_crop.mean(axis=(0, 1))
    #         goalkeeper_team_id = int(np.argmin([np.linalg.norm(gk_color - ref) for ref in reference_colors]))
    if goalkeeper_detections.xyxy is not None and len(goalkeeper_detections.xyxy) > 0:
    gx1, gy1, gx2, gy2 = map(int, goalkeeper_detections.xyxy[0])
    gk_crop = frame[gy1:gy2, gx1:gx2]
    if gk_crop.size != 0:
        image_pil = Image.fromarray(gk_crop)
        inputs = clip_processor(text=team_prompts, images=image_pil, return_tensors="pt", padding=True)
        with torch.no_grad():
            outputs = clip_model(**inputs)
            probs = outputs.logits_per_image.softmax(dim=1)
            goalkeeper_team_id = int(torch.argmax(probs).item())



    labels = [f"#{tracker_id}" for tracker_id in all_detections.tracker_id]

    annotated_frame = frame.copy()
    # annotated_frame = ellipse_annotator.annotate(scene=annotated_frame, detections=all_detections)
    team_color_map = {
    0: sv.Color.from_hex('#f8d4dc'),  # light pink for team 0
    1: sv.Color.from_hex('#c2c1ef')   # light purple-blue for team 1
    }

    # Annotate players with team-specific colors
    for i, det in enumerate(all_detections):
      team_id = team_assignments.get(int(det.tracker_id), 0)
      player_color = team_color_map.get(team_id, sv.Color.from_hex('#cccccc'))
      single_annotator = sv.EllipseAnnotator(color=player_color, thickness=2)
      annotated_frame = single_annotator.annotate(scene=annotated_frame, detections=all_detections[i:i+1])

    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=all_detections, labels=labels)
    annotated_frame = triangle_annotator.annotate(scene=annotated_frame, detections=ball_detections)

    # annotated_frame = ellipse_annotator.annotate(scene=annotated_frame, detections=goalkeeper_detections)
    # Annotate the goalkeeper with a custom color based on team_id
    goalkeeper_team_color_map = {
      0: sv.Color.from_hex('#d3e4f4'),  # Icy Blue
      1: sv.Color.from_hex('#d5c6e0')   # Dusty Lavender
    }

    if goalkeeper_detections.xyxy is not None and len(goalkeeper_detections.xyxy) > 0:

      gk_color = goalkeeper_team_color_map.get(goalkeeper_team_id, sv.Color.from_hex('#cccccc'))
      gk_annotator = sv.EllipseAnnotator(color=gk_color, thickness=2)
      annotated_frame = gk_annotator.annotate(scene=annotated_frame, detections=goalkeeper_detections)


    writer.write(annotated_frame)

    frame_log = {
        "frame": frame_index,
        "timestamp_sec": round(frame_index / fps, 2),
        "detections": []
    }

    for tracker_id, xyxy in zip(all_detections.tracker_id, all_detections.xyxy):
        frame_log["detections"].append({
            "label": "player",
            "id": int(tracker_id),
            "team_id": team_assignments.get(int(tracker_id), -1), #Assign Team ID
            "box": list(map(float, xyxy))
        })

    for xyxy in ball_detections.xyxy:
        frame_log["detections"].append({
            "label": "ball",
            "id": None,
            "box": list(map(float, xyxy))
        })

    for xyxy in goalkeeper_detections.xyxy:
        frame_log["detections"].append({
            "label": "GK",
            "id": None,
            "team_id": goalkeeper_team_id, #Assign Goalkeeper ID
            "box": list(map(float, xyxy))
        })

    log_data.append(frame_log)

cap.release()
writer.release()

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(log_data, f, indent=2)

print(f"\n✅ Annotated video saved to: {OUTPUT_VIDEO_PATH}")
print(f"✅ Detection log saved to: {JSON_OUTPUT_PATH}")

NameError: name 'sv' is not defined

Possession Calculation

In [ ]:
import json
import numpy as np

# Load detection log
with open("121364_0_log.json", "r") as f:
    log_data = json.load(f)

# Define possession threshold (in pixels, tweak as needed)
possession_threshold = 50

# Loop through each frame and compute possession
for frame in log_data:
    ball_box = None
    players = []

    for det in frame["detections"]:
        if det["label"] == "ball":
            ball_box = det["box"]
        elif det["label"] == "player":
            players.append({"id": det["id"], "box": det["box"]})

    possessor_id = None
    min_distance = float("inf")

    if ball_box:
        ball_center = np.array([
            (ball_box[0] + ball_box[2]) / 2,
            (ball_box[1] + ball_box[3]) / 2
        ])

        for player in players:
            player_box = player["box"]
            player_center = np.array([
                (player_box[0] + player_box[2]) / 2,
                (player_box[1] + player_box[3]) / 2
            ])
            distance = np.linalg.norm(player_center - ball_center)

            if distance < possession_threshold and distance < min_distance:
                min_distance = distance
                possessor_id = player["id"]

    frame["possession"] = possessor_id  # Will be None if no one has the ball

# Save updated log
output_path = "121364_0_log_with_possession.json"
with open(output_path, "w") as f:
    json.dump(log_data, f, indent=2)

print(f"✅ Possession data added and saved to: {output_path}")


What to do next:


Add the other features that you see fit/needed from (https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/football-ai.ipynb#scrollTo=8ixZlM06Gmae)


